In [1]:
from termcolor import colored
import numpy as np
import random
import copy
import math
import sys

In [2]:
ROW_COUNT = 6
COLUMN_COUNT = 10

PLAYER1 = 0
PLAYER2 = 1

EMPTY = 0
PLAYER1_PIECE = 1
PLAYER2_PIECE = 2

WINDOW_LENGTH = 5
all_scores = []

In [3]:
def create_board():
    board = np.zeros((ROW_COUNT, COLUMN_COUNT))
    return board


def print_board(board):
    flip_board = np.flip(board, 0)
    print()
    
    for r in range(ROW_COUNT):
        for c in range(COLUMN_COUNT):
            
            if flip_board[r][c] == PLAYER1_PIECE:
                print(colored(int(flip_board[r][c]), 'grey', 'on_red'), end=' ')
                
            elif flip_board[r][c] == PLAYER2_PIECE:
                print(colored(int(flip_board[r][c]), 'grey', 'on_blue'), end=' ')
                
            else:
                print(colored(int(flip_board[r][c]), 'grey'), end=' ')
                
        print()


# Logic
def get_next_open_row(board, col):
    for r in range(ROW_COUNT):
        if board[r][col] == 0:
            return r


def drop_piece(board, row, col, piece):
    board[row][col] = piece


def is_valid_location(board, col):
    if col != None:
        return board[ROW_COUNT - 1][col] == 0
    return False


def get_valid_locations(board):
    valid_locations = []
    for col in range(COLUMN_COUNT):
        if is_valid_location(board, col):
            valid_locations.append(col)
    return valid_locations


def is_terminal_node(board):
    return winning_move(board, PLAYER1_PIECE) or winning_move(board, PLAYER2_PIECE) or \
           len(get_valid_locations(board)) == 0


def winning_move(board, piece):
    # Check horizontal locations for win
    for c in range(COLUMN_COUNT - 4):
        for r in range(ROW_COUNT):
            if board[r][c] == piece and board[r][c + 1] == piece and board[r][c + 2] == piece and \
                    board[r][c + 3] == piece and board[r][c + 4] == piece:
                return True

    # Check vertical locations for win
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT - 4):
            if board[r][c] == piece and board[r + 1][c] == piece and board[r + 2][c] == piece and \
                    board[r + 3][c] == piece and board[r + 4][c] == piece:
                return True

    # Check positively sloped diaganols
    for c in range(COLUMN_COUNT - 4):
        for r in range(ROW_COUNT - 4):
            if board[r][c] == piece and board[r + 1][c + 1] == piece and board[r + 2][c + 2] == piece and \
                    board[r + 3][c + 3] == piece and board[r + 4][c + 4] == piece:
                return True
            
    # Check negatively sloped diaganols
    for c in range(COLUMN_COUNT - 4):
        for r in range(4, ROW_COUNT):
            if board[r][c] == piece and board[r - 1][c + 1] == piece and board[r - 2][c + 2] == piece and \
                    board[r - 3][c + 3] == piece and board[r - 4][c + 4] == piece:
                return True
            
    return False

In [4]:
def evaluate_window(window):
    score = 0
    window = list(window)
    if window.count(PLAYER1_PIECE) == 5:
        score += 2000
    elif window.count(PLAYER1_PIECE) == 4 and window.count(EMPTY) == 1:
        score += 4
    elif window.count(PLAYER1_PIECE) == 3 and window.count(EMPTY) == 2:
        score += 3
    elif window.count(PLAYER1_PIECE) == 2 and window.count(EMPTY) == 3:
        score += 2
    elif window.count(PLAYER1_PIECE) == 1 and window.count(EMPTY) == 4:
        score += 1

    if window.count(PLAYER2_PIECE) == 5:
        score -= 2000
    elif window.count(PLAYER2_PIECE) == 4 and window.count(EMPTY) == 1:
        score -= 4
    elif window.count(PLAYER2_PIECE) == 3 and window.count(EMPTY) == 2:
        score -= 3
    elif window.count(PLAYER2_PIECE) == 2 and window.count(EMPTY) == 3:
        score -= 2
    elif window.count(PLAYER2_PIECE) == 1 and window.count(EMPTY) == 4:
        score -= 1
    return score

def score_position(board):
    score = 0
    for r in range(ROW_COUNT):
        for c in range(COLUMN_COUNT - 4):
            window = [board[r][c], board[r][c + 1], board[r][c + 2], board[r][c + 3], board[r][c + 4]]
            score += evaluate_window(window)
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT - 4):
            window = [board[r][c], board[r + 1][c], board[r + 2][c], board[r + 3][c], board[r + 4][c]]
            score += evaluate_window(window)
    for r in range(ROW_COUNT - 4):
        for c in range(COLUMN_COUNT - 4):
            window = list(
                [board[r][c], board[r + 1][c + 1], board[r + 2][c + 2], board[r + 3][c + 3], board[r + 4][c + 4]])
            score += evaluate_window(window)
    for r in range(ROW_COUNT - 4):
        for c in range(COLUMN_COUNT - 4):
            window = [board[r][c], board[r - 1][c + 1], board[r - 2][c + 2], board[r - 3][c + 3], board[r - 4][c + 4]]
            score += evaluate_window(window)
    return score

In [5]:
def maximizer(board, depth):
    if is_terminal_node(board) or depth == 0:
        return score_position(board)
    value = -999999999
    cols = get_valid_locations(board)
    for col in cols:
        board_copy = copy.copy(board)
        drop_piece(board_copy, get_next_open_row(board, col), col, PLAYER1_PIECE)
        value = max(value, minimizer(board_copy, depth - 1))
    return value
    
def minimizer(board, depth):
    if is_terminal_node(board) or depth == 0:
        return score_position(board)
    value = 999999999
    cols = get_valid_locations(board)
    for col in cols:
        board_copy = copy.copy(board)
        drop_piece(board_copy, get_next_open_row(board, col) , col, PLAYER2_PIECE)
        value = min(value, maximizer(board_copy, depth - 1))

    return value
    
def minimax_get_action(board, depth, player):
    scores = []
    cols = get_valid_locations(board)
    if player == PLAYER1:
        for col in range(10):
            if cols.count(col) != 1:
                scores.append(-999999999)
                continue
            board_copy = copy.copy(board)
            drop_piece(board_copy, get_next_open_row(board, col), col, PLAYER1_PIECE)
            scores.append(minimizer(board_copy, depth))
        return np.where(scores == np.max(scores))[0][0]
    else:
        for col in range(10):
            if cols.count(col) != 1:
                scores.append(999999999)
                continue
            board_copy = copy.copy(board)
            drop_piece(board_copy, get_next_open_row(board, col), col, PLAYER2_PIECE)
            scores.append(maximizer(board_copy, depth))
        return np.where(scores == np.min(scores))[0][0]

In [6]:
def maximizer_alpha_beta(board, depth):
    if is_terminal_node(board) or depth == 0:
        return score_position(board)
    value = -999999999
    cols = get_valid_locations(board)
    for col in cols:
        board_copy = copy.copy(board)
        drop_piece(board_copy, get_next_open_row(board, col), col, PLAYER1_PIECE)
        value = max(value, minimizer_alpha_beta(board_copy, depth - 1))
        if len(all_scores) != 0:
            if value > min(all_scores):
                return value
    return value

def minimizer_alpha_beta(board, depth):
    if is_terminal_node(board) or depth == 0:
        return score_position(board)
    value = 999999999
    cols = get_valid_locations(board)
    for col in cols:
        board_copy = copy.copy(board)
        drop_piece(board_copy, get_next_open_row(board, col), col, PLAYER2_PIECE)
        value = min(value, maximizer_alpha_beta(board_copy, depth - 1))
        if len(all_scores) != 0:
            if value < max(all_scores):
                return value
    return value
    
def minimax_alpha_beta_get_action(board, depth, player):
    scores = []
    cols = get_valid_locations(board)
    if player == PLAYER1:
        for col in range(10):
            if cols.count(col) != 1:
                scores.append(-999999999)
                continue
            board_copy = copy.copy(board)
            drop_piece(board_copy, get_next_open_row(board, col), col, PLAYER1_PIECE)
            scores.append(minimizer_alpha_beta(board_copy, depth))
        all_scores.append(np.max(scores))
        return np.where(scores == np.max(scores))[0][0]
    else:
        for col in range(10):
            if cols.count(col) != 1:
                scores.append(999999999)
                continue
            board_copy = copy.copy(board)
            drop_piece(board_copy, get_next_open_row(board, col), col, PLAYER2_PIECE)
            scores.append(maximizer_alpha_beta(board_copy, depth))
        all_scores.append(np.min(scores))
        return np.where(scores == np.min(scores))[0][0]

In [17]:
board = create_board()
print_board(board)
game_over = False
turn = random.randint(PLAYER1, PLAYER2)
while not game_over:
    if len(get_valid_locations(board)) == 0:
        label = 'TIE!'
        game_over = True    
    if turn == PLAYER1 and not game_over:
        col = random.choice(get_valid_locations(board))
#         col = minimax_get_action(board, 2, PLAYER1)
#         col = minimax_alpha_beta_get_action(board, 2, PLAYER1)
#         col = int(input('Please enter the column you want to drop in (0 to 9): '))
#         while not col in get_valid_locations(board):
#             col = int(input('Please enter the column you want to drop in (0 to 9): '))
        if is_valid_location(board, col):
            row = get_next_open_row(board, col)
            drop_piece(board, row, col, PLAYER1_PIECE)
            if winning_move(board, PLAYER1_PIECE):
                label = 'Player 1 wins!'
                game_over = True
            turn = 1 - turn
            print_board(board)
    if turn == PLAYER2 and not game_over:
        col = random.choice(get_valid_locations(board))
#         col = minimax_get_action(board, 2, PLAYER2)
#         col = minimax_alpha_beta_get_action(board, 2, PLAYER2)
#         col = int(input('Please enter the column you want to drop in (0 to 9): '))
#         while not col in get_valid_locations(board):
#             col = int(input('Please enter the column you want to drop in (0 to 9): '))

        if is_valid_location(board, col):
            row = get_next_open_row(board, col)
            drop_piece(board, row, col, PLAYER2_PIECE)
            if winning_move(board, PLAYER2_PIECE):
                label = 'Player 2 wins!'
                game_over = True
            turn = 1 - turn
            print_board(board)
    if game_over:
        label = label + ' Game over'
        print ('\n', colored(label, 'green'))


0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 

0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 2 0 0 0 0 0 0 0 0 

0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 1 0 0 0 0 0 0 0 0 
0 2 0 0 0 0 0 0 0 0 

0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 1 0 0 0 0 0 0 0 0 
0 2 0 0 0 0 0 0 0 2 

0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 1 0 0 0 0 0 0 0 0 
0 2 0 0 0 0 1 0 0 2 

0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 1 0 0 0 0 0 0 0 0 
0 2 0 2 0 0 1 0 0 2 

0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 1 0 0 0 0 0 0 0 1 
0 2 0 2 0 0 1 0 0 2 

0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 2 
0 1 0 0 0 0 0 0 0 1 
0 2 0